In [56]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from scipy.stats import ttest_ind

In [2]:
premise_file = "master_set.csv"
premise_df = pd.read_csv(premise_file)
seven_11 = "Rotten_Tomatoes_07-11.csv"
RT_711_df = pd.read_csv(seven_11)
fandango_df = pd.read_csv('Fandango_2015.csv')
RT_100_df = pd.read_csv("rotten_tomatoes_100.csv")
RT_t10_df = pd.read_csv("top_10_movies.csv")

In [3]:

master = premise_df[['movie_title','title_year','director_name','genres','content_rating',
                     'country', 'budget','gross','imdb_score']]

In [4]:
master = master.rename(columns={'movie_title':'movie_title','title_year':'year','director_name':'director','genres':'genre',
                       'content_rating':'content_rating',
                     'country':'country', 'budget':'budget','gross':'gross_box_office'})

In [5]:
master= master.fillna(0)


In [6]:
master[['year', 'budget','gross_box_office']] = master[['year', 'budget','gross_box_office']].astype(int)

In [7]:
# master.set_index("Movie_Title", inplace=True)

master.head(1)

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score
0,Batman v Superman: Dawn of Justice,2016,Zack Snyder,Action|Adventure|Sci-Fi,PG-13,USA,250000000,330249062,6.9


In [8]:
#strip the \xao off of each movie

master['movie_title'] = master['movie_title'].str.rstrip('\xa0')

# master = master.dropna(axis = 0,how = 'any', inplace = True)


In [9]:
def ok():
    #return(master['Movie Title'][0]) # has \xa0 on the end
     #return(RT_711_df['Movie Title'][10]) # has a space on the end
#     return(fandango_df['Movie Title'][0]) # good
#     return(RT_100_df['Movie Title'][0]) # good
#     return(RT_t10_df['Movie Title'][0]) # good
    ok()
    
master["movie_title"] = master.movie_title.drop_duplicates()


In [10]:
master = master[pd.notnull(master["movie_title"])]
master.head(1)

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score
0,Batman v Superman: Dawn of Justice,2016,Zack Snyder,Action|Adventure|Sci-Fi,PG-13,USA,250000000,330249062,6.9


In [11]:
master.set_index('movie_title', inplace=True)
master.head(1)

,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score
movie_title,,,,,,,,
Batman v Superman: Dawn of Justice,2016,Zack Snyder,Action|Adventure|Sci-Fi,PG-13,USA,250000000,330249062,6.9


In [12]:
master[master.index.isnull()].count()


year                0
director            0
genre               0
content_rating      0
country             0
budget              0
gross_box_office    0
imdb_score          0
dtype: int64

In [13]:
master.index.value_counts()

Domestic Disturbance                                  1
WarGames                                              1
The Tale of Despereaux                                1
The Scorpion King                                     1
Karachi se Lahore                                     1
Envy                                                  1
Sex and the City 2                                    1
Echo Dr.                                              1
Day of the Dead                                       1
Shipwrecked                                           1
Dry Spell                                             1
Splash                                                1
The Other End of the Line                             1
Eye of the Dolphin                                    1
The Whole Ten Yards                                   1
28 Weeks Later                                        1
Breaking Upwards                                      1
Tremors                                         

In [14]:
# seven_11 = "Rotten_Tomatoes_07-11.csv"
# RT_711_df = pd.read_csv(seven_11)

In [15]:
RT_711_df = RT_711_df.rename(columns = {'Film':'movie_title','Rotten Tomatoes Ratings %': 'rotten_tomatoes_score',
                                        'Audience Ratings %': 'audience_score'})

In [16]:
RT_711_df["movie_title"] = RT_711_df["movie_title"].drop_duplicates()

In [17]:
RT_711_df['movie_title'] = RT_711_df['movie_title'].str.rstrip(' ')
RT_711_df = RT_711_df[['movie_title', "rotten_tomatoes_score", "audience_score"]]
RT_711_df.set_index('movie_title', inplace=True)
RT_711_df.head()

,rotten_tomatoes_score,audience_score
movie_title,,
(500) Days of Summer,87,81
"10,000 B.C.",9,44
12 Rounds,30,52
127 Hours,93,84
17 Again,55,70


In [18]:
# fandango 2015

# fandango_df = pd.read_csv('Fandango_2015.csv')

In [19]:
fandango_df.columns

Index(['FILM', 'RottenTomatoes', 'RottenTomatoes_User', 'Metacritic',
       'Metacritic_User', 'IMDB', 'Fandango_Stars', 'Fandango_Ratingvalue',
       'RT_norm', 'RT_user_norm', 'Metacritic_norm', 'Metacritic_user_nom',
       'IMDB_norm', 'RT_norm_round', 'RT_user_norm_round',
       'Metacritic_norm_round', 'Metacritic_user_norm_round',
       'IMDB_norm_round', 'Metacritic_user_vote_count', 'IMDB_user_vote_count',
       'Fandango_votes', 'Fandango_Difference'],
      dtype='object')

In [20]:
fandango_df = fandango_df.rename(columns = {'FILM':'movie_title','RottenTomatoes': 'rotten_tomatoes_score',
                                            'RottenTomatoes_User':'audience_score'})
fandango_df['movie_title'] = fandango_df['movie_title'].str[:-6]


In [21]:
fandango_df["movie_title"] = fandango_df["movie_title"].drop_duplicates()

In [22]:
fandango_df = fandango_df[['movie_title', 'rotten_tomatoes_score', "audience_score"]]
fandango_df.set_index('movie_title', inplace=True)
fandango_df.head()

,rotten_tomatoes_score,audience_score
movie_title,,
Avengers: Age of Ultron,74,86
Cinderella,85,80
Ant-Man,80,90
Do You Believe?,18,84
Hot Tub Time Machine 2,14,28


In [23]:
# top 100 movies

RT_100_df = pd.read_csv("rotten_tomatoes_100.csv")

In [24]:
RT_100_df.head(1)


,Rank,Title,RatingTomatometer,No. of Reviews,Genres
0,1,Black Panther (2018),97,444,action|adventure


In [25]:
# for index, row in RT_100_df.iterrows():
#     if row['movie_title'] == np.nan:
#         RT_100_df.drop(index, inplace=True)
# RT_100_df.reset_index()
RT_100_df = RT_100_df.drop(columns=["Rank"])


In [26]:
RT_100_df= RT_100_df.rename(columns = {'Title':'movie_title','RatingTomatometer':'rotten_tomatoes_score'})

In [27]:
RT_100_df = RT_100_df[['movie_title','rotten_tomatoes_score']]
RT_100_df.head()

,movie_title,rotten_tomatoes_score
0,Black Panther (2018),97
1,Mad Max: Fury Road (2015),97
2,Wonder Woman (2017),93
3,Metropolis (1927),99
4,Coco (2017),97


In [28]:
RT_100_df["movie_title"] = RT_100_df["movie_title"].drop_duplicates()

In [29]:
RT_100_df = RT_100_df[:101]
RT_100_df

,movie_title,rotten_tomatoes_score
0,Black Panther (2018),97
1,Mad Max: Fury Road (2015),97
2,Wonder Woman (2017),93
3,Metropolis (1927),99
4,Coco (2017),97
5,Dunkirk (2017),92
6,Incredibles 2 (2018),94
7,Star Wars: The Last Jedi (2017),91
8,Logan (2017),93
9,Star Wars: Episode VII - The Force Awakens (2015),93


In [30]:
RT_100_df['movie_title'] = RT_100_df['movie_title'].str[:-6]
RT_100_df.set_index('movie_title', inplace=True)


In [31]:
#top 10 movies form each year

RT_t10_df = pd.read_csv("top_10_movies.csv")

In [32]:
RT_t10_df = RT_t10_df.rename(columns = {'title':'movie_title','rt_freshness':'rotten_tomatoes_score',
                                      'audience_freshness':'audience_score' })
RT_t10_df = RT_t10_df[pd.notnull(RT_t10_df["movie_title"])]
RT_t10_df = RT_t10_df[pd.notnull(RT_t10_df["rotten_tomatoes_score"])]
RT_t10_df = RT_t10_df[pd.notnull(RT_t10_df["audience_score"])]


In [33]:
RT_t10_df = RT_t10_df.loc[RT_t10_df['movie_title'].notnull(),:]

In [34]:
RT_t10_df = RT_t10_df[['movie_title', 'rotten_tomatoes_score', 'audience_score']]
RT_t10_df.dropna(subset = ['movie_title'],how="any",inplace = True)
RT_t10_df["movie_title"] = RT_t10_df["movie_title"].drop_duplicates()
RT_t10_df[['rotten_tomatoes_score','audience_score']]=RT_t10_df[['rotten_tomatoes_score','audience_score']].astype(int)
RT_t10_df.set_index('movie_title', inplace=True)
RT_t10_df.head()

,rotten_tomatoes_score,audience_score
movie_title,,
Captain America: The Winter Soldier,89,92
Dawn of the Planet of the Apes,90,89
Guardians of the Galaxy,91,93
Interstellar,72,86
Maleficent,49,71


In [35]:
# get rid of NaN values in the RT_t10_df

import numpy as np

# for row in RT_t10_df:
#     if (math.isnan(row.index)):

RT_t10_df[RT_t10_df.index != np.nan]
          
#RT_t10_df[RT_t10_df.index.isnull()]

RT_t10_df = RT_t10_df.reset_index()

for index, row in RT_t10_df.iterrows():
    if row['movie_title'] == np.nan:
        RT_t10_df.drop(index, inplace=True)


In [36]:
RT_t10_df = RT_t10_df.drop(383, axis = 0)

In [37]:
RT_t10_df = RT_t10_df.drop(397, axis = 0)

In [38]:
def mights_work():
    return(RT_t10_df[RT_t10_df.movie_title.isnull()])
mights_work()

,movie_title,rotten_tomatoes_score,audience_score


In [39]:
RT_t10_df = RT_t10_df.set_index('movie_title')

In [40]:
def might_work():
    return(RT_t10_df[RT_t10_df.index.isnull()])
might_work()

,rotten_tomatoes_score,audience_score
movie_title,,


In [41]:
rds_connection_string = "postgres:postgres@localhost:5432/Movies"
engine = create_engine(f'postgresql://{rds_connection_string}')
# Confirm tables
engine.table_names()

['master', 'rt_711_df', 'fandango_df', 'rt_100_df', 'rt_t10_df']

In [42]:
master.to_sql(name='master', con=engine, if_exists='append', index=True)

In [43]:
RT_711_df.to_sql(name='rt_711_df', con=engine, if_exists='append', index=True)
RT_t10_df.to_sql(name='rt_t10_df', con=engine, if_exists='append', index=True)
fandango_df.to_sql(name='fandango_df', con=engine, if_exists='append', index=True)
RT_100_df.to_sql(name='rt_100_df', con=engine, if_exists='append', index=True)

In [45]:
master_from_sql_df = pd.read_sql_query("select * from master", con = engine).head()
master_from_sql_df

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score
0,Batman v Superman: Dawn of Justice,2016,Zack Snyder,Action|Adventure|Sci-Fi,PG-13,USA,250000000,330249062,6.9
1,Captain America: Civil War,2016,Anthony Russo,Action|Adventure|Sci-Fi,PG-13,USA,250000000,407197282,8.2
2,Star Trek Beyond,2016,Justin Lin,Action|Adventure|Sci-Fi|Thriller,PG-13,USA,185000000,130468626,7.5
3,The Legend of Tarzan,2016,David Yates,Action|Adventure|Drama|Romance,PG-13,USA,180000000,124051759,6.6
4,X-Men: Apocalypse,2016,Bryan Singer,Action|Adventure|Sci-Fi,PG-13,USA,178000000,154985087,7.3


In [54]:
#master combined with rt_711_df

master_combined = pd.read_sql_query("""select m.movie_title, m.year, m.director, m.genre, m.content_rating,
m.country,m.budget,m.gross_box_office,m.imdb_score,rt_7.rotten_tomatoes_score,
rt_7.audience_score from master as m join rt_711_df as rt_7 on m.movie_title = rt_7.movie_title """, con = engine)
master_combined.head(1)

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score,rotten_tomatoes_score,audience_score
0,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,Action|Adventure|Fantasy,PG-13,USA,250000000,241063875,6.7,34.0,61.0


In [70]:
rotten_master = pd.read_sql_query("""select * from rt_711_df union select * from rt_t10_df union select * from fandango_df 
union select *, null as audience_score from rt_100_df""", con = engine)
rotten_master.head()

,movie_title,rotten_tomatoes_score,audience_score
0,Tangled,90.0,87.0
1,Gremlins,85.0,77.0
2,Pirates of the Caribbean: Dead Man's Chest,54.0,72.0
3,Rush Hour 3,20.0,68.0
4,Penelope,52.0,74.0


In [74]:
master_master =  master_combined.merge(rotten_master,on = 'movie_title', how = 'left')
master_master.head()

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score,rotten_tomatoes_score_x,audience_score_x,rotten_tomatoes_score_y,audience_score_y
0,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,Action|Adventure|Fantasy,PG-13,USA,250000000,241063875,6.7,34.0,61.0,33.0,54.0
1,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,Action|Adventure|Fantasy,PG-13,USA,250000000,241063875,6.7,34.0,61.0,34.0,61.0
2,Green Lantern,2011,Martin Campbell,Action|Adventure|Sci-Fi,PG-13,USA,200000000,116593191,5.6,27.0,48.0,27.0,48.0
3,Transformers: Dark of the Moon,2011,Michael Bay,Action|Adventure|Sci-Fi,PG-13,USA,195000000,352358779,6.3,35.0,67.0,35.0,56.0
4,Transformers: Dark of the Moon,2011,Michael Bay,Action|Adventure|Sci-Fi,PG-13,USA,195000000,352358779,6.3,35.0,67.0,35.0,67.0


In [78]:
master_master['rotten_tomatoes_score'] = (master_master['rotten_tomatoes_score_x'] +master_master['rotten_tomatoes_score_y'])/2

In [80]:
master_master['audience_score'] = (master_master['audience_score_x'] +master_master['audience_score_y'])/2

In [82]:
master_master = master_master[['movie_title','year','director','genre','content_rating','country','budget','gross_box_office','imdb_score','rotten_tomatoes_score','audience_score' ]]

In [84]:
master_master.head()

,movie_title,year,director,genre,content_rating,country,budget,gross_box_office,imdb_score,rotten_tomatoes_score,audience_score
0,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,Action|Adventure|Fantasy,PG-13,USA,250000000,241063875,6.7,33.5,57.5
1,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,Action|Adventure|Fantasy,PG-13,USA,250000000,241063875,6.7,34.0,61.0
2,Green Lantern,2011,Martin Campbell,Action|Adventure|Sci-Fi,PG-13,USA,200000000,116593191,5.6,27.0,48.0
3,Transformers: Dark of the Moon,2011,Michael Bay,Action|Adventure|Sci-Fi,PG-13,USA,195000000,352358779,6.3,35.0,61.5
4,Transformers: Dark of the Moon,2011,Michael Bay,Action|Adventure|Sci-Fi,PG-13,USA,195000000,352358779,6.3,35.0,67.0
